## Noise watermarking - salt and pepper

In [ ]:
import os
import sys; sys.path.insert(0, "..")

from dotenv import load_dotenv
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pickle
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

from src.constants import IMG_SQUARE_SIDE, LABELS_LIST
from src.model.vgg16 import create_vgg16_model
from src.watermarking.content.noise.saltpepper import SaltpepperNoiseWatermarker
from src.watermarking.utils import VGG16Preprocessor, WeatherLoader

In [ ]:
load_dotenv()
DATASET_FOLDER = os.getenv("DATASET_FOLDER")
MARKER_FOLDER = os.getenv("MARKER_FOLDER")
SAVE_FOLDER = os.path.join(os.getcwd(), os.pardir, "saved_objects")

In [ ]:
with open(os.path.join(SAVE_FOLDER, "img_df.pkl"), "rb") as handle:
    img_df = pickle.load(handle)

Préparer les données

In [ ]:
# Initialiser
# ✏️ à compléter

# Preparer le dataset standard
X_origin = np.array(
    img_df["image_path"].apply(weather_loader.load).apply(vgg16_preprocessor.preprocess).tolist()
)
y_origin = to_categorical(img_df["label"])

# Séparer le dataset standard
X_macro_train_origin, X_test_origin, y_macro_train_origin, y_test_origin = train_test_split(
    X_origin,
    y_origin,
    stratify=y_origin,
    test_size=0.2,
    random_state=0,
)
X_train_origin, X_val_origin, y_train_origin, y_val_origin = train_test_split(
    X_macro_train_origin, 
    y_macro_train_origin,
    stratify=y_macro_train_origin,
    test_size=0.2,
    random_state=0,
)


# Marquer le dataset
X_wtm, y_wtm = ... # ✏️ à compléter
indices = ... # ✏️ à compléter

# Séparer le dataset marqué
X_macro_train_wtm, X_test_wtm, y_macro_train_wtm, y_test_wtm, idx_macro_train, idx_test = train_test_split(
    X_wtm,
    y_wtm,
    indices,
    test_size=0.2,
    random_state=0,
)
X_train_wtm, X_val_wtm, y_train_wtm, y_val_wtm = train_test_split(
    X_macro_train_wtm, 
    y_macro_train_wtm,
    test_size=0.2,
    random_state=0,
)

# Concatener les datasets
X_train_full = np.concatenate([X_train_origin, X_train_wtm])
y_train_full = np.concatenate([y_train_origin, y_train_wtm])
X_val_full = np.concatenate([X_val_origin, X_val_wtm])
y_val_full = np.concatenate([y_val_origin, y_val_wtm])
X_test_full = np.concatenate([X_test_origin, X_test_wtm])
y_test_full = np.concatenate([y_test_origin, y_test_wtm])

Afficher les 5 premières images du je de test filigrané.

In [ ]:
from PIL import Image

def show_original_images_from_index(df, indices, n=3):
    plt.figure(figsize=(12, 6))
    for i, idx in enumerate(indices[:n]):
        img_path = df["image_path"].iloc[idx]
        img = Image.open(img_path)
        plt.subplot(1, n, i+1)
        plt.imshow(img)
        plt.axis("off")
    plt.show()

show_original_images_from_index(img_df, idx_test, n=3)

In [ ]:
show_images_with_pred(
    df=img_df,
    indices=idx_test,
    y_pred=y_val_pred,       # one-hot
    y_true=y_test_wtm,   # one-hot
    n=5
)

Model preparation

In [ ]:
model_save_path = os.path.join(SAVE_FOLDER, "watermarked_model_best_weights.h5")
checkpoint = ModelCheckpoint(
    model_save_path,
    monitor="val_loss",
    verbose=True,
    save_best_only=True,
    mode="min",
)
es = EarlyStopping(monitor="val_loss", mode="min", verbose=True, patience=5)
callbacks = [checkpoint, es]

In [ ]:
with tf.device("/cpu:0"):
    model = create_vgg16_model()
    history = model.fit(
        X_train_full,
        y_train_full,
        epochs=1,
        batch_size=64,
        callbacks=callbacks,
        validation_data=(X_val_full, y_val_full),
        verbose=True,
    )

Compléter le code de la fonction d'affichage pour afficher les images avec les prédictions du modèle filigrané, et appeler cette nouvelle fonction au même échantillon que précédemment.

In [ ]:
# ✏️ Votre code ici

Visualiser/anaylser les sorties.

In [ ]:
# ✏️ Votre code ici